# Intro To Serialization and Pickler Combinators

While the overall purpose of this series is to cover the creation of a poker agent, this particular segment serves an useful purpose on its own. More than just for poker, if you've ever had trouble writing code to serialize data from some game state to the kinds of floating point tensors that neural nets need to digest, read this tutorial study the code for a bit and those troubles will be a thing of the past.

Pickler combinators are one of the grand functional programming inventions on par with parser combinators that are sadly overlooked by the wider programming community. They work well enough for Python as well.

To start things off, we'll define a PU class to serve as an abstract interface for its inheritors. It will serve to document the two functions Pickler/Unpickler classes need to implement.

In [55]:
from typing import Literal, Tuple

class PU:
    def pickle(self,x,i : int,ar) -> None: 
        """
        Serialized the object 'x' by setting the relevant fields of 'ar' to 1.
        
        x - Composite object to be serialized.

        i - Starting position of array 'ar'.

        ar - An 1d array. Can be anything satisfying the array interface constraints.
        """
        raise NotImplementedError()
        
    def unpickle(self,i : int,ar) -> Tuple[object,Literal[1, 0]]:
        """
        Deserializes the object from the array 'ar'. Returns the deserialized part
        of the object for that particular unpickler and 1/0 whether the unpickling has succeeded.

        x - Composite object to be serialized.

        i - Starting position of array 'ar'.
        """
        raise NotImplementedError()

The abstract interface above is what all the pickler combinators will follow and is mostly there for documentation purposes. We need a primitive combinator for integers, and then composite combinators for tuples and for union types. And since union types might have cases with no fields, it is easiest to start with the union combinator.

In [56]:
def serialize(schema : PU,x):
    ar = [0] * schema.size
    schema.pickle(x,0,ar)
    return ar

def deserialize(schema : PU,ar):
    x,c = schema.unpickle(0,ar)
    if c != 1: raise Exception("Invalid format.")
    return x

Before that, here are the helpers to actually run the combinators. `serialize` takes in the combinator and the input and produces the pickled binary array. `deserialize` does the opposite.

In [57]:
def round_trip_assert(schema : PU,x):
    assert x == deserialize(schema,serialize(schema,x)), "PU round trip failed."

 

An easy way to make sure that the serializers work is to do a round trip, serializing and deserializing them back and comparing the values.

In [58]:
class Unit(PU):
    def __init__(self) -> None:
        super().__init__()
        self.size = 1
        
    def pickle(self,x : Tuple[()],i,ar): 
        assert x == (), "The input to this should be a an empty tuple."
        ar[i] = 1

    def unpickle(self,i,ar):
        x = ar[i]
        assert x == 0 or x == 1, "Unpickling failure. The unit type should always be either be active or inactive."
        return (), int(x)

In [59]:
serialize(Unit(),())

[1]

This combinator isn't very interesting as it just produces the [1] array.

In [60]:
round_trip_assert(Unit(),())

Since running the above does not raise an exception, we have some basic assurance that the serialization is correct.

The next will be a PU combinator for representing integers in the one-hot format.

In [61]:
class Int(PU):
    def __init__(self,size : int) -> None:
        super().__init__()
        self.size = size

    def pickle(self,x : int,i,ar):
        assert 0 <= x < self.size, f'Pickling failure. Int value out of bounds. Got: {x} Size: {self.size}'
        ar[i + x] = 1
        
    def unpickle(self,i_start,ar):
        case,c = 0,0
        for i in range(i_start,i_start+self.size):
            x = ar[i]
            assert x == 0 or x == 1, "Unpickling failure. The int type must either be active or inactive."
            if x == 1: case,c = i,c+1
        assert c == 0 or c == 1, "Unpickling failure. Too many active indices in the one-hot vector."
        return case-i_start,c

During serialization, the int combinator sets the array to 1 at the given index plus the element. Deserialization is a bit harder as it needs to find the index of that 1 by iterating over all the elements. Since we need to account for the union types, it could also be that case that all the elements are zero. Lastly we need to account for the possibility of the array having trash data and assert that all elements are either 0 or 1.

This combinator has interesting functionality. 

In [62]:
print(serialize(Int(5),1))
print(serialize(Int(7),5))

[0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 1, 0]


In [63]:
round_trip_assert(Int(5),3)
round_trip_assert(Int(5),1)

No exception gets raised. Serializing integers to the one hot format is the simplest kind of serialization, but it is very useful. For the poker games, I use it for card ranks and suits for example. It could be used for stack size if they are sufficiently small, but for those it would be better to have a proper binary serializer. Here it is.

In [64]:
class BinInt(PU):
    """
    Unlike int which stores the value as a one-hot vector, this one stores it as a binary value.
    The value 0 is stored as 1, 1 as 2 and so on. During unpickling the zero vector is considered inactive.
    The max value is taken up by that inactive vactor. Hence the actual capacity is
    `2 ** size - 1` rather than `2 ** size`.
    """
    def __init__(self,size : int) -> None:
        super().__init__()
        assert 1 <= size <= 62, "The field size has to be in the [1,62] range."
        self.x_nearTo = 2 ** size - 1
        self.size = size

    def pickle(self,x : int,i_start,ar):
        assert 0 <= x < self.x_nearTo, f'Pickle failure. Bin int value out of bounds. Got: {x} Size: {self.x_nearTo}'
        x += 1
        for i in range(self.size):
            q = 1 << (self.size - i - 1)
            ar[i_start + i] = x // q
            x %= q
            
    def unpickle(self,i_start,ar):
        r = 0
        for i in range(self.size):
            x = ar[i_start+i]
            assert x == 0 or x == 1, "Unpickling failure. The bin int type must either be active or inactive."
            r += int(x) << (self.size - i - 1)
        return r-1, (1 if 0 < r else 0)

This one is similar to the Int serializer, except it uses the binary rather than the one-hot format and so is exponentially more efficient in the size of the input element.

Here are is an example of it in use.

In [65]:
[serialize(BinInt(10),i) for i in [0,1,5,123,654]]

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
 [1, 0, 1, 0, 0, 0, 1, 1, 1, 1]]

In [66]:
for i in [0,1,5,123,654]: round_trip_assert(BinInt(10),i)

The round trip test works fine. The next combinator is the first composite one.

In [67]:
class Tuple(PU):
    def __init__(self,*pus) -> None:
        super().__init__()
        self.size = 0
        for x in pus: self.size += x.size
        self.pus = pus

    def pickle(self,xs,i,ar):
         for x,pu in zip(xs,self.pus): 
            pu.pickle(x,i,ar)
            i += pu.size
            
    def unpickle(self,i,ar):
        c,l = 0,[]
        for pu in self.pus: 
            x,c2 = pu.unpickle(i,ar)
            i += pu.size
            c += c2
            l.append(x)
        assert c == 0 or c == len(self.pus), "Unpickling failure. The tuple should have all of its elements be either active or inactive."
        return tuple(l), 0 if c == 0 else 1

What this does is allows PU combinators to be composed as tuples. As all datatypes can be mapped to either tuples or unions once we have those two, we will have unfetted freedom of serialization.

Its use is fairly simple.

In [68]:
serialize(Tuple(Int(5),BinInt(5),Unit()),(2,29,()))

[0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1]

Here is a more motivated example.

In [69]:
suit = Int(4)
rank = Int(13)
card = Tuple(suit,rank)
stack = BinInt(7)
print(serialize(Tuple(stack,card,card),(100,(0,0),(1,0))))

[1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


Naturally, tuple combinators can be composed with other tuple combinators as shown in the above example. That is what makes them powerful.

Before moving on to unions, one combinator worth covering is the array combinator. In poker, different betting streets have a different number of cards on the table and it would be inefficient to use unions to cover those eventualities.

In [70]:
class Array(PU):
    def __init__(self,pu : PU,n : int) -> None:
        """
        pu - PU for the elements of the array.

        n - The maximum size of the array.
        """
        super().__init__()
        self.pu, self.n = pu, n
        self.size = 1 + self.pu.size * n # The extra field is the emptiness flag which differentiates empty from inactive arrays.

    def pickle(self,xs,i,ar):
        assert len(xs) <= self.n, "Pickling failure. The given array is too large."
        if len(xs) == 0: ar[i] = 1
        i += 1
        for x in xs: 
            self.pu.pickle(x,i,ar)
            i += self.pu.size
            
    def unpickle(self,i,ar):
        c_empty = ar[i]
        assert c_empty == 0 or c_empty == 1, "Unpickling failure. The array emptiness flag should be 1 or 0."
        c_empty = int(c_empty)
        i += 1

        xs = []
        for j in range(self.n):
            x,c_sub = self.pu.unpickle(i,ar)
            if j == len(xs):
                if c_sub == 1: xs.append(x) 
            else:
                assert c_sub == 0, "Unpickling failure. Expected an inactive subsequence in the array unpickler."
            i += self.pu.size
        assert not (c_empty == 1 and 0 < len(xs)), "Unpickling failure. Empty arrays should not have active elements."
        return xs, c_empty | min(1,len(xs))

The array combinator needs some extra complexity to differentiate between inactive and empty arrays. During unpickling also needs to check that after the first inactive elements, no more active elements come up.

Here is an example of them in action.

In [71]:
suit = Int(4)
rank = Int(13)
card = Tuple(suit,rank)
stack = BinInt(7)
board = Array(card,5)
print(serialize(Tuple(stack,card,card,board),(100,(0,0),(1,0),[(2,0),(3,0),(1,5),(2,2),(0,11)])))

[1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]


The number of cards passed into the array can be varied between 0 and 5 in the above example.

The next is the union combinator.

In [113]:
class Union(PU):
    def __init__(self,pus) -> None:
        super().__init__()
        assert 0 < len(pus), "The number of cases in `pus` should be greater than zero."
        self.pus = pus
        self.size = 0
        for ins,pu in pus: self.size += pu.size

    def pickle(self,x,i,ar):
        for ins,pu in self.pus:
            if isinstance(x,ins):
                pu.pickle(x.data,i,ar)
                return
            i += pu.size
        raise Exception("The input does not match the schema.")

    def unpickle(self,i,ar):
        r,c = None,0
        for ins,pu in self.pus:
            x,c2 = pu.unpickle(i,ar)
            assert not (c == 1 and c2 == 1), "Only one case of the union should be active."
            if c2 == 1: r = ins(x); c = 1
            i += pu.size
        return r,c

In [117]:
class UnionData:
    def __init__(self,data) -> None: self.data = data
    def __eq__(self, o: object) -> bool: return self.data == o.data

class Action(UnionData): pass
class Fold(UnionData): pass
class Call(UnionData): pass
class Raise(UnionData): pass
class Card(UnionData): pass

stack_size = 10
action = Union(((Fold,Unit()),(Call,Unit()),(Raise,Int(stack_size+1))))
suit = Int(1)
rank = Int(3)
card = Tuple(suit,rank)
observation = Union(((Action,action),(Card,card)))
observations = Array(observation,5)

args = observations,[Card((0,0)),Card((0,1)),Action(Call(())),Action(Call(())),Action(Raise(2))]
print(serialize(*args))
round_trip_assert(*args)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
